# Tutorial "Calculate_SRF.py"

Authors: [Asmae KHARRAB](mailto:asmae.kharrab@insa-lyon.fr), [Florian MERINDOL](mailto:florian.merindol@insa-lyon.fr)

[INSA Lyon](https://www.insa-lyon.fr), France, 08/10/2021

## Introduction

In order to weight criterias by SRF method (Simos-Roy-Figueira), this Python code [**Calculate_SRF.py**](Calculate_SRF.py), displayed below, allows to automate calculations from Excel file where user had classified previously criterias. 

## SRF method presentation

The Simos-Roy-Figueira (SRF) method is an Simos method improvement, so-called "method of cards"

The cards method allow to weight criterias. The decision maker chooses the relative position of cards, representing criterias, by classifying them under pyramidal way. He can add white cards to indicate an additional importance between two successive ranks. Visuel and concrete, that technique is often used as communication way. A mathematical processing allows to wieght each criteria.  
However, that method had limits, unseen for the user. 

Firstly, the weighting isn't normalized systematically (the sum of weight must be equal to one). 

Then, the criteria visual position is modified by mathematical processing. Two cards on the last rank (number 1) have an position evaluated at 1,5. This mathematical deformation reduces the weight list which can be calculated. 

Finally, the relative importance between the fist and the last criteria is imposed. That induces strong disparities in the weighting when the number of criterias is high. 

The SRF method improves the mathematical processing of criterias, with adddition of Z ratio, representing the relative importance noted previously. 

## Python code presentation

The different steps of the SRF method code are presented here.  

The modules panda and openpyxl are used to extract datas from an excel file. 

In [1]:
import pandas as pd

## 1. First step : Definition of functions

#### 1.1 File recovering
The first step is to recover datas from a given Excel file, that is composed in five sheets : 
    
    1. 'Critères' : presentation of criterias with their indexs, descriptions, interests and evaluation ways. 
    2. 'Pondération Critères Systèmes' : the users classifies the system criterias and gives a Z ratio
    3. 'Pondération Critères Enveloppe' :  the users classifies the covering criterias and gives a Z ratio
    4. 'Calculs' : the calculation steps, here automated by the Python code, are realized. 
    5. 'Résultats' : the results are displayed. 
    
The Z ratio is the relative importance between the first and the last criteria. 

In the following function, 'recuperate_file', the 'name' of Excel file is indicated, with the sheet number to extract. The datas are returned in list to facilitate the data processing. 

In [2]:
def recuperate_file(name, index_sheet):
    """To recuperate a sheet from Excel file to make into list"""
    feuille_recup = pd.read_excel(name, sheet_name=index_sheet)
    return feuille_recup.values.tolist()

#### 1.2 Dictionary definition
The correspondence between the criteria name and its index is made by the functions dico_. A function is writen for each field. 

Warning ! The used index are valid only for the 'Pondération Critères.xlsm' file. That limits this SRF method, and could be improved by coding the choice process, realised for the moment in Excel. 

In [3]:
def fdico_economie(feuille):
    """ Economy dictionary definition, containing correspondence between index and criteria """
    dico_economie = {}
    for x in range(7,15):
        dico_economie[(feuille[x][2])] = (feuille[x][1])
    return dico_economie

def fdico_technique(feuille):
    """ Technique dictionary definition, containing correspondence between index and criteria """
    dico_technique = {}
    for x in range(18, 22):
        dico_technique[(feuille[x][2])] = (feuille[x][1])
    return dico_technique

def fdico_social(feuille):
    """ Social dictionary definition, containing correspondence between index and criteria """
    dico_social = {}
    for x in range(25,30):
        dico_social[(feuille[x][2])] = (feuille[x][1])
    return dico_social

def fdico_environnement(feuille):
    """ Environment dictionary definition, containing correspondence between index and criteria """
    dico_environnement = {}
    for x in range(33,40):
        dico_environnement[(feuille[x][2])] = (feuille[x][1])
    return dico_environnement

#### 1.3 Recovering of user's choices
The user's choices are recuperated in form of list. For that, the function tableau_choix receives the sheet 'feuille' where are the datas to recover, and the index of table lines. 

In [4]:
def tableau_choix(feuille,dico,index):
    """Recovering of user's choice in form of list. Return the Z ratio too"""
    tableau_choix_environnement = [[0 for j in range(4)] for i in range(15)]
    a = index[0]
    b = index[1]
    for x in range(a,b):
        tableau_choix_environnement[x - a][0]= x - a+1
        for y in range(2,5):
            tableau_choix_environnement[x - a][y - 1]=str(feuille[x][y])
            if (dico.get(tableau_choix_environnement[x - a][y - 1], 1) != 1) :
                tableau_choix_environnement[x - a][y - 1] = dico.get(tableau_choix_environnement[x - a][y - 1])
    return tableau_choix_environnement, feuille[index[1]+1][2]

#### 1.4 Definition of calculation functions
The lines below present the calculation functions used to determinate the SRF method parameters. 

Firstly, the function tableau_choix return a matrix containing criterias, classified by users. A first function allows to delate lines useless for ranking of criteria. 

In [5]:
def traitement(l):
   """Allow to withdraw useless white cards"""
   while l[-1][1] == 'nan':
       del l[-1]
   return l

A second method allows to assign numbering for each rank, conversely of ranking. The last criteria is assigned the number one.  

In [6]:
def calcul_rang(l):
   listnouveaurang = [None]*len(l)
   for i in range(len(l)):
       listnouveaurang[i] = l[len(l)-i-1]
       listnouveaurang[i][0]=i+1
   return listnouveaurang

The calcul_n method allows to calculate the ranking number in the given choice matrix. For each line not empty, the n parameter is incremented. 

In [7]:
def calcul_n(l):
   n = 0
   for i in range(len(l)):
       for j in range(1, len(l[i])):
           if l[i][j] != 'nan':
               n +=1
   return n

This calcul_er_prime function allows to determinate the number of white cards  in two successive ranks. 
If the line is empty (with 'nan'), the matrix 'l' adds 1, else adds 0 white card. The 'n' number corresponds with white cards number of previous rank

In [8]:
def calcul_er_prime(l,n):
    for i in range(len(l)-1):
        if l[i+1][1] == 'nan':
            l[i].append(1)
        else:
            l[i].append(0)
    if n == 0:
        l[len(l)-1].append(1)
    else:
        l[len(l)-1].append(0)

    for i in range(1,len(l)):
        if l[len(l)-i-1][-1] != 0:
            l[len(l) - i - 1][-1] = l[len(l)-i-1][-1] + l[len(l)-i][-1]
    for i in range(len(l)):
        if l[i][1] == 'nan' and l[i][2] == 'nan' and l[i][3] == 'nan':
            l[i][-1] = 0
    return l

The calcul_er method allows to calculate the 'er' parameter from the er' list, calculated previously. 

In [9]:
def calcul_er(l):
    e = 0
    for i in range(len(l)):
        l[i].append(l[i][-1]+1)
        e += l[i][-1]
    return l,e

The calcul_u method allows to calculate the 'u' parameter from the 'er' list, calculated previously. . 

In [10]:
def calcul_u(z,e):
    u = (z-1)/e
    return u

The calcul_k-i-prime method allows to calculate the ki' parameter. 

In [11]:
def calcul_k_i_prime(u,l):
    somme_er = 0
    for i in range(len(l)):
        if l[i][1] != 'nan':
            l[i].append(1+u*somme_er)
            somme_er += l[i][-2]
        else:
            l[i].append('nan')
            somme_er += l[i][-2]
    return l

The 'traitement2' method allows to create a table associating criteria and ki' corresponding. 

In [12]:
def traitement2(l):
    """Allow to obtain table with criteria and ki' corresponding"""
    liste_traitee = []
    for i in range(len(l)):
        for j in range(1 , 4):
            if l[i][j] != 'nan':
                liste_traitee.append([l[i][j],l[i][-1]])
    return liste_traitee

Once the table of [criteria, ki'] got, the ki' sum is calculated.  

In [13]:
def calcul_K_prime(l):
  K_prime = 0
  for i in range(len(l)):
      K_prime += l[i][1]
  return K_prime

The ki*, corresponding to final weighting not rounding, is calculated.  

In [14]:
def calcul_ki_etoile(l,K_prime):
  for i in range(len(l)):
      l[i].append((100/K_prime)*l[i][1])
  return l

A mathematical processing is made to normalize the ki*. For that, the ki" is calculated by ki* truncation.  

In [15]:
def calcul_ki_2prime(l):
  for i in range(len(l)):
      l[i].append(int(l[i][2]*100)/100)
  return l

The ki" is calculated. 

In [16]:
def calcul_K_2prime(l):
  K_2prime = 0
  for i in range(len(l)):
      K_2prime += l[i][3]
  return K_2prime

The final weighting is calculated then. For that, the operators [di, M, L, F] are used. The w parameter is defined to choose the decimal number. Finally, the F+ and F- matrices indicate if the weight should be rounded up or rounded down. 

In [17]:
def calcul_ki(l,K_2prime,n):
  w = 2
  for i in range(len(l)):
      l[i].append((10 ** (-w) - (l[i][2] - l[i][3])) / l[i][2])  # Calcul di
      l[i].append((l[i][2] - l[i][3]) / l[i][2])  # Calcul di_barre
      if l[i][4] > l[i][5]: #Calcul M
          l[i].append(1)
      else:
          l[i].append(0)
      if l[i][6] == 1: #Calcul di_M
          l[i].append(l[i][4])
      else:
          l[i].append('-')

  v = int(10 ** (w) * (100 - K_2prime))  # Calcul v
  m = 0  # Calcul m
  for i in range(len(l)):
      m += l[i][6]

  L = [] # Calcul L
  for i in range(len(l)):
      if l[i][7] != '-':
          L.append(l[i][7])
  L.sort()
  if L == []:
      L = [0] * n

  vmn = v + m - n
  if m+v > n:
      for i in range(vmn,len(L)):
          L[i] = 0
  else:
      L = [0]*n

  #Calcul flèche L
  for i in range(len(L)):
      compteur_di = 0
      if L[i] == l[0][4]:
          compteur_di +=1
  if compteur_di != 0:
          l[0].append(1)
  else:
          l[0].append(0)
  vmn =  v + m -n
  for i in range(1,len(l)):
      compteur_di = 0
      for j in range(len(L)):
          if L[j] == l[i][4]:
              compteur_di +=1
      if compteur_di != 0:
          s=0
          j = 0
          while j != i and i < len(L):
              s+=L[j]
              j +=1
          if s>vmn :
              l[i].append(0)
          else:
              l[i].append(1)

      else:
          l[i].append(0)


  for i in range(len(l)): #Calcul dibarre_différent_M
      if l[i][6] == 0:
          l[i].append(l[i][5])
      else:
          l[i].append('-')

  Lbarre = []  # Calcul Lbarre
  nvm = n - v -m
  for i in range(len(l)):
      if l[i][9] != '-':
          Lbarre.append(l[i][9])
  Lbarre.sort()
  if Lbarre == []:
      Lbarre = [0] * n

  if m + v < n:
      for i in range(nvm, len(Lbarre)):
          Lbarre[i] = 0
  else:
      Lbarre = [0] * n


  #Calcul flèche Lbarre

  for i in range(len(Lbarre)):
      compteur_dibarre = 0
      if Lbarre[i] == l[0][5]:
          compteur_di += 1
  if compteur_di != 0:
      l[0].append(1)
  else:
      l[0].append(0)

  for i in range(1, len(l)):
      compteur_dibarre = 0
      for j in range(len(Lbarre)):
          if Lbarre[j] == l[i][5]:
              compteur_dibarre += 1

      if compteur_dibarre != 0:
          s = 0
          j = 0
          while j != i and i < len(Lbarre):
              s += Lbarre[j]
              j +=1
          if s > nvm:
              l[i].append(0)
          else:
              l[i].append(1)
      else:
          l[i].append(0)
  #Calcul F+ / F-
  for i in range(len(l)):  # F+
      if m + v < n:
          conditionF = n - v - m
          if l[i][6] == 0:
              l[i].append(1)
          else:
              l[i].append(0)
      else:
          conditionF = n - v
  fbis = [0] * len(l)
  verif = 0
  j = n-1
  while verif < conditionF and j!=-1:
      if l[j][10]==1 and l[j][6]==0:
          fbis[j] = 0
          if verif <= nvm:
              l[j][11] = fbis[j]
              verif = verif +1
      j = j-1

  # Calcul F- puis ki
  for i in range(len(l)):
      if l[i][11] == 0: #F-
          l[i].append(1)
      else:
          l[i].append(0)
      if l[i][12] == 1: #ki
          l[i].append(int(l[i][2]*(10**w))/10**w)
      else:
          l[i].append(int(l[i][2]*(10**w)+1)/(10**w))
  veri = 0
  for i in range(len(l)):
      veri += l[i][13]
  return l,veri

## 2. Second step : Simos-Roy-Figueira method

#### 2.1 Data recovering
The first step is to recover the user's choices from the 'Pondération Critères.xlsm' file. For that, the methods (presented parts 1.1 and 1.2) are used. 

The sheets '1', '2' and '3' are recuperated. A warning message is displayed to indicate that the Openpyxl module doesn't support the 'Data Validation' datas from Excel. Our datas are not concerned. 

In [18]:
feuille1 = recuperate_file('Pondération critères.xlsm', 0)
feuille2 = recuperate_file('Pondération critères.xlsm', 1)
feuille3 = recuperate_file('Pondération critères.xlsm', 2)

C:\Users\flori\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


The dictionarys associating criterias and index are created. 

In [19]:
dico_economie = fdico_economie(feuille1)
dico_environnement = fdico_environnement(feuille1)
dico_social = fdico_social(feuille1)
dico_technique = fdico_technique(feuille1)

The indexes allowing the identification of data range in the calculation sheed are given. 

In [20]:
index_environnement = [92,107]
index_economie = [21,36]
index_social = [69,84]
index_technique = [45,60]

The tables containing the criterias ranking are created.  

In [21]:
tableau_choix_environnement, z_environnement = tableau_choix(feuille2,dico_environnement,index_environnement)
tableau_choix_economie, z_economie = tableau_choix(feuille2,dico_economie,index_economie)
tableau_choix_technique, z_technique = tableau_choix(feuille2,dico_technique,index_technique)
tableau_choix_social, z_social = tableau_choix(feuille2,dico_social,index_social)

#### 2.2 Data processing
The second step allows to process datas to determinate the criteria weights with the functions defined in part 1.3

In [22]:
tableau = traitement(tableau_choix_economie)

tableau = calcul_rang(tableau)

n = calcul_n(tableau)

tableau = calcul_er_prime(tableau,n)

tableau,e = calcul_er(tableau)

u = calcul_u(z_economie,e)


tableau = calcul_k_i_prime(u,tableau)

tableau = traitement2(tableau)


K_prime = calcul_K_prime(tableau)

tableau = calcul_ki_etoile(tableau,K_prime)

tableau = calcul_ki_2prime(tableau)

K_2prime = calcul_K_2prime(tableau)

tableau,total = calcul_ki(tableau,K_2prime,n)

Finally, the criteria associated with its weight is displayed. 

In [23]:
for i in range(0,len(tableau)):
    print(str(tableau[i][0])+" "+str(tableau[i][13]))
print("Somme des poids = "+str(total))

g1.4 2.37
g1.3 9.21
g1.2 10.18
g1.5 11.16
g1.8 14.09
g1.7 17.01
g1.1 17.99
g1.6 17.99
Somme des poids = 100.0
